<h1>Capstone</h1>
This project will use location data to explore different neighborhoods.  
Created by Daniel Ochoa


In [2]:
import pandas as pd
import numpy as np


/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
import urllib.request
import lxml
url_pc_canada = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
with urllib.request.urlopen(url_pc_canada) as response:
   html = response.read()

df = pd.read_html(html,skiprows=1)[0]
# Add column names
df.columns = ['postal_code', 'borough', 'neighborhood']


In [4]:
#drop not assigned borough's
df = df[~df['borough'].isin(['Not assigned'])]

In [5]:
postal_data = pd.read_csv('http://cocl.us/Geospatial_data')
postal_data = pd.DataFrame(postal_data)
postal_data = postal_data.rename(columns={"Postal Code": "postal_code"})



In [11]:
df_data = pd.merge(df, postal_data, left_on='postal_code', right_on='postal_code')

In [12]:
#cluster 
# set number of clusters
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


kclusters = 7
# run k-means clustering

cluster_lat_long = df_data[['Latitude','Longitude']]
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_lat_long)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


df_data['cluster'] = kmeans.labels_
df_data.head()

,postal_code,borough,neighborhood,Latitude,Longitude,cluster
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,5
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,5
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,5
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,5


In [10]:
# Visualize.
# create map

#This map creates 7 clusters based of lat and long proximity. 
#each dot marks a postal_code

start_lat = '43.806686'
start_long= '-79.194353'
map_clusters = folium.Map(location=[start_lat, start_long], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_data['Latitude'], df_data['Longitude'], df_data['cluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters